In [21]:
import numpy as np

class Bernoulli(object):
    def wj(self, X, y, i):
        features = len(X.getnnz(axis=0))
        X = X.toarray()
        X = (X > 0).astype(np.int_)
        y = [1 if element == i else 0 for element in y]
        y = np.array(y)
        wj1 = []
        wj0 = []
        for j in range(features):
                theta_10 = (X[y==0][j].sum()+1) / (float((y==0).sum())+2)
                theta_11 = (X[y==1][j].sum()+1) / (float((y==1).sum())+2)
                wj1.append(np.log((theta_11)/(theta_10)))
                wj0.append(np.log((1-theta_11)/(1-theta_10)))
        return wj1, wj0

    def prob_y1(self,y, i):
        y = [1 if element == i else 0 for element in y]
        y = np.asarray(y)
        return y[y==1].sum() / float(len(y))

    def prob_y0(self, y, i):
        return 1-prob_y1(y, i)

    def fit(self, X, y, X_test):
        deci = []
        for i in range(1,21):
            save = []
            wj1, wj0 = self.wj(X, y, i)
            p1 = prob_y1(y, i)
            p0 = prob_y0(y, i)
            w0 = np.log(p1/p0) + np.sum(wj0)
            w = np.subtract(wj1, wj0)
            for j in range(len(X_test.getnnz(axis=1))):
                wc = float(w*X_test[j].T)
                save.append(float(wc + w0))
            deci.append(np.asarray(save))
        return deci

    def predict(self,X, y, X_test):
        deci = self.fit(X, y, X_test)
        pred = []
        for i in range(len(X_test.getnnz(axis=1))):
            prob = []
            for j in range(20):
                prob.append(deci[j][i])
            maxProb = max(prob)
            for j in range(20):
                if deci[j][i] == maxProb:
                    pred.append(j+1)
        pred = np.array(pred)
        return pred
    
    import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

reddit_data = pd.read_csv("reddit_train.csv") #train data
df = pd.DataFrame(reddit_data) #train data
reddit_data_copy = reddit_data.copy() #train data
reddit_data_copy = reddit_data_copy.drop(columns=["subreddits"]) #train data
X = reddit_data_copy[reddit_data_copy.columns[1]]  #raw comments

train_comments_list = []
for i in range(len(X.index)):
    train_comments_list.append(X.iloc[i])
train_comments_list = list(train_comments_list)
train_comments_list = np.array(train_comments_list)


Y = reddit_data[reddit_data.columns[-1]]
y_modified = []         #raw classes
train_subreddits_class = []    #important, class in integer
for i in range(len(Y.index)):
    y_modified.append(Y.loc[i])
for i in range(len(y_modified)):
    if y_modified[i] == 'hockey':
        train_subreddits_class.append(1)
    elif y_modified[i] == 'nba':
        train_subreddits_class.append(2)
    elif y_modified[i] == 'leagueoflegends':
        train_subreddits_class.append(3)
    elif y_modified[i] == 'soccer':
        train_subreddits_class.append(4)
    elif y_modified[i] == 'funny':
        train_subreddits_class.append(5)
    elif y_modified[i] == 'movies':
        train_subreddits_class.append(6)
    elif y_modified[i] == 'Overwatch':
        train_subreddits_class.append(7)
    elif y_modified[i] == 'anime':
        train_subreddits_class.append(8)
    elif y_modified[i] == 'trees':
        train_subreddits_class.append(9)
    elif y_modified[i] == 'GlobalOffensive':
        train_subreddits_class.append(10)
    elif y_modified[i] == 'nfl':
        train_subreddits_class.append(11)
    elif y_modified[i] == 'AskReddit':
        train_subreddits_class.append(12)
    elif y_modified[i] == 'gameofthrones':
        train_subreddits_class.append(13)
    elif y_modified[i] == 'conspiracy':
        train_subreddits_class.append(14)
    elif y_modified[i] == 'worldnews':
        train_subreddits_class.append(15)
    elif y_modified[i] == 'wow':
        train_subreddits_class.append(16)
    elif y_modified[i] == 'europe':
        train_subreddits_class.append(17)
    elif y_modified[i] == 'canada':
        train_subreddits_class.append(18)
    elif y_modified[i] == 'Music':
        train_subreddits_class.append(19)
    elif y_modified[i] == 'baseball':
        train_subreddits_class.append(20)
train_subreddits_class = np.array(train_subreddits_class)
X_train = train_comments_list
y_train = train_subreddits_class

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import WordPunctTokenizer
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
import re

#preprocessing 
lem = WordNetLemmatizer()
tknzr = WordPunctTokenizer()
stem = PorterStemmer()
replace_by_space = re.compile('[/(){}\[\]\|@,;!.#><?""''%$^*`:]')
stop_words = set(stopwords.words('english'))
unused_char = re.compile('[^0-9a-zA-Z]')

for i in range(len(X_train)):
    X_train[i] = X_train[i].lower()
    X_train[i] = replace_by_space.sub(' ', X_train[i])
    X_train[i] = unused_char.sub(' ', X_train[i])
    X_train[i] = ' '.join(lem.lemmatize(word) for word in X_train[i].split() if word not in stop_words)

tf_idf_vectorizer = CountVectorizer(stop_words = set(stopwords.words('english')), max_df = 1.0, ngram_range=(1, 3), min_df = 0.000005, analyzer='word',binary=True, tokenizer = WordPunctTokenizer().tokenize)
vectors_train_idf = tf_idf_vectorizer.fit_transform(X_train)

/Users/yoooooooh/Desktop/Anaconda3/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'"] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [7]:
from sklearn.feature_selection import SelectKBest, chi2, f_classif, f_regression, mutual_info_classif
skb = SelectKBest(chi2, k=10000)
vectors_train_idf_normalized_selected = skb.fit_transform(vectors_train_nomalized, y_train)

(70000, 10000)


In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(vectors_train_idf_normalized_selected, y_train, train_size=0.9, test_size=0.1)

(63000, 10000) (63000,)
(7000, 10000) (7000,)


In [9]:
y_train = np.asarray(y_train)
y_test = np.asarray(y_test)

(63000, 10000) (63000,)
(7000, 10000) (7000,)


In [ ]:
import timeit

start = timeit.default_timer()
cl = Bernoulli()
pred = cl.predict(X_train, y_train, X_test)
stop = timeit.default_timer()

print('Time: ', stop - start) 

In [ ]:
pred